In [115]:
import numpy as np
import pandas as pd

In [116]:
credits_df=pd.read_csv("credits.csv")
movies_df=pd.read_csv("movies.csv")

In [ ]:
credits_df

In [ ]:
movies_df

In [119]:
movies_df=movies_df.merge(credits_df,on='title')

In [ ]:
movies_df.info()

In [121]:
movies_df=movies_df[['movie_id','title','overview','genres','keywords','crew']]

In [ ]:
movies_df.head()

In [ ]:
movies_df.info()

In [ ]:
movies_df.isnull().sum()

In [125]:
movies_df.dropna(inplace=True)   # dropna will drop all missing values from original dataset

In [ ]:
movies_df.duplicated().sum()

In [ ]:
movies_df.iloc[0].genres

In [128]:
import ast #abstract syntax tree

In [129]:
def convert (obj):
  L=[]
  for i in ast.literal_eval(obj):
    L.append(i['name'])
  return L        # to convert dataset into structured way - genres & keyword

In [130]:
movies_df['genres']=movies_df['genres'].apply(convert)
movies_df['keywords']=movies_df['keywords'].apply(convert)

In [131]:
def convert3(obj):
  L=[]
  counter=0
  for i in ast.literal_eval(obj):
    if counter !=3:
      L.append(i['name'])
      counter +=1
    else:
      break
    return L    # to convert dataset into structured way - cast

In [71]:
#movies_df['cast']=movies_df['cast'].apply(convert3)

In [132]:
def fetch_director(obj):
  L=[]
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
      L.append(i['name'])
  return L    # to convert dataset into structured way - crew

In [133]:
movies_df['crew'] = movies_df['crew'].apply(fetch_director)

In [134]:
movies_df['overview'] = movies_df['overview'].apply(lambda x:x.split())   # to separate each word from sentence

In [135]:
movies_df['genres']=movies_df['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [136]:
movies_df['keywords']=movies_df['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [105]:
#movies_df['cast'] = movies_df['cast'].apply(lambda x: [i.replace(" ", "") if isinstance(i, str) else i for i in x])

In [137]:
movies_df['crew']=movies_df['crew'].apply(lambda x:[i.replace(" ","") for i in x])    # to remove space for beteer recommendation

In [138]:
movies_df['tag']=movies_df['overview']+movies_df['genres']+movies_df['keywords']+movies_df['crew']

In [139]:
new_df=movies_df[['movie_id','title','tag']]

In [ ]:
new_df['tag']=new_df['tag'].apply(lambda x:' '.join(x))   #remove square[] brackets

In [ ]:
new_df['tag']=new_df['tag'].apply(lambda X:X.lower())     #convert to lower case for better prediction

In [142]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')
tfidf = TfidfVectorizer(stop_words='english')

In [ ]:
cv.fit_transform(new_df['tag']).toarray().shape

In [ ]:
vectors = tfidf.fit_transform(new_df['tag']).toarray()
vectors[0]

In [ ]:
len(cv.get_feature_names_out())

In [147]:
import nltk
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

In [148]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_df['tag'] = new_df['tag'].apply(stem)

In [150]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity(vectors)

In [ ]:
cosine_similarity(vectors).shape

In [153]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity[0]

In [ ]:
similarity[0].shape

In [ ]:
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x:x[1])[1:6]

In [157]:
def recommend(movie):
    movie_index = new_df[new_df['title'].apply(lambda x:x.lower())==movie.lower()].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]

    for i in movie_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('Iron Man')

In [ ]:
recommend('Avatar')

In [ ]:
recommend('Aliens')